# Analyse des Décisions avec 27 Critères

**Objectif :** Montrer que la **solution 1** est meilleure que les trois autres solutions (2, 3 et 4) en utilisant des explications de type trade-off.

In [13]:
from gurobipy import *
import pandas as pd

## 1. Chargement des Données

In [14]:
# Lecture du fichier Excel
df = pd.read_excel('data27crit.xlsx', header=None)

# Extraction des noms de critères (ligne 5, colonnes 1-27)
criteria = df.iloc[4, 1:28].tolist()
print(f"Critères ({len(criteria)}) : {criteria}")

# Extraction des poids (ligne 6, colonnes 1-27)
weights_list = df.iloc[5, 1:28].tolist()
weights = {criteria[i]: weights_list[i] for i in range(len(criteria))}
print(f"\nPoids : {weights}")

# Extraction des solutions (lignes 8-11, colonnes 1-27)
solutions = {}
for i, sol_name in enumerate(['solution 1', 'solution 2', 'solution 3', 'solution 4']):
    values = df.iloc[7 + i, 1:28].tolist()
    solutions[sol_name] = {criteria[j]: values[j] for j in range(len(criteria))}

print(f"\nSolutions chargées : {list(solutions.keys())}")
print(f"\nSolution 1 (échantillon) : a={solutions['solution 1']['a']}, b={solutions['solution 1']['b']}, ...")

Critères (27) : ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A']

Poids : {'a': 2, 'b': 1, 'c': 5, 'd': 3, 'e': 4, 'f': 6, 'g': 1, 'h': 7, 'i': 4, 'j': 3, 'k': 8, 'l': 5, 'm': 1, 'n': 2, 'o': 1, 'p': 4, 'q': 5, 'r': 6, 's': 2, 't': 7, 'u': 1, 'v': 2, 'w': 6, 'x': 3, 'y': 5, 'z': 5, 'A': 1}

Solutions chargées : ['solution 1', 'solution 2', 'solution 3', 'solution 4']

Solution 1 (échantillon) : a=532, b=120, ...


## 2. Fonctions d'Explication Trade-Off

Les différents types d'explications :
- **(1-1)** : Un Pro compense exactement un Con
- **(1-m)** : Un Pro compense plusieurs Cons
- **(m-1)** : Plusieurs Pros compensent un seul Con
- **Mixte** : Combinaison de (1-m) et (m-1)

In [15]:
def calculate_deltas(sol1_name, sol2_name):
    """Calcule les contributions pondérées (deltas) entre deux solutions."""
    sol1 = solutions[sol1_name]
    sol2 = solutions[sol2_name]
    
    deltas = {}
    for k in criteria:
        deltas[k] = weights[k] * (sol1[k] - sol2[k])
    
    pros = [k for k, v in deltas.items() if v > 0]
    cons = [k for k, v in deltas.items() if v < 0]
    
    return deltas, pros, cons

In [16]:
def explain_1_1(sol1_name, sol2_name):
    """Explication de type (1-1) : chaque Pro compense exactement un Con."""
    deltas, pros, cons = calculate_deltas(sol1_name, sol2_name)
    
    print(f"\n{'='*60}")
    print(f"EXPLICATION (1-1) : {sol1_name} > {sol2_name}")
    print(f"{'='*60}")
    print(f"Arguments Pour (Pros) : {len(pros)} critères")
    print(f"Arguments Contre (Cons) : {len(cons)} critères")
    
    # Paires valides (delta_p + delta_q > 0)
    valid_pairs = [(p, q) for p in pros for q in cons if deltas[p] + deltas[q] > 0]
    print(f"Paires (1-1) valides : {len(valid_pairs)}")
    
    if len(valid_pairs) == 0:
        print("\nAucune paire valide - Explication (1-1) impossible.")
        return False, None
    
    # Modélisation Gurobi
    m = Model("Explication_1_1")
    
    c = {}
    for p, q in valid_pairs:
        c[(p, q)] = m.addVar(vtype=GRB.BINARY, name=f"c_{p}_{q}")
    
    m.update()
    
    # Contrainte 1 : Chaque Con couvert par exactement un Pro
    for q in cons:
        m.addConstr(quicksum(c[(p, q)] for p in pros if (p, q) in valid_pairs) == 1, name=f"Cover_{q}")
    
    # Contrainte 2 : Chaque Pro utilisé au plus une fois
    for p in pros:
        m.addConstr(quicksum(c[(p, q)] for q in cons if (p, q) in valid_pairs) <= 1, name=f"Use_{p}")
    
    m.setObjective(0, GRB.MINIMIZE)
    m.params.outputflag = 0
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        print("\nEXPLICATION (1-1) TROUVÉE :")
        explanation = []
        for p, q in valid_pairs:
            if c[(p, q)].x > 0.5:
                net = deltas[p] + deltas[q]
                explanation.append((p, q, deltas[p], deltas[q], net))
                print(f"   • Trade-off ({p},{q}) : +{deltas[p]:.1f} + ({deltas[q]:.1f}) = +{net:.1f}")
        return True, explanation
    else:
        print("\nExplication (1-1) INFAISABLE.")
        return False, None

In [17]:
def explain_1_m(sol1_name, sol2_name):
    """Explication de type (1-m) : un Pro peut compenser plusieurs Cons."""
    deltas, pros, cons = calculate_deltas(sol1_name, sol2_name)
    
    print(f"\n{'='*60}")
    print(f"EXPLICATION (1-m) : {sol1_name} > {sol2_name}")
    print(f"{'='*60}")
    print(f"Arguments Pour (Pros) : {len(pros)} critères")
    print(f"Arguments Contre (Cons) : {len(cons)} critères")
    
    if len(pros) == 0 or len(cons) == 0:
        if len(cons) == 0:
            print("\n✅ Aucun argument contre ! Solution 1 domine trivialement.")
            return True, "dominance"
        print("\nAucun argument pour - Explication impossible.")
        return False, None
    
    m = Model("Explication_1_m")
    
    x = m.addVars(pros, cons, vtype=GRB.BINARY, name="x")
    m.update()
    
    # Chaque Con couvert par exactement un Pro
    for c in cons:
        m.addConstr(quicksum(x[p, c] for p in pros) == 1, name=f"Cover_{c}")
    
    # Validité du trade-off pour chaque Pro
    epsilon = 0.001
    for p in pros:
        m.addConstr(deltas[p] + quicksum(deltas[c] * x[p, c] for c in cons) >= epsilon, name=f"Valid_{p}")
    
    m.setObjective(0, GRB.MINIMIZE)
    m.params.outputflag = 0
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        print("\nEXPLICATION (1-m) TROUVÉE :")
        explanation = []
        for p in pros:
            covered = [c for c in cons if x[p, c].x > 0.5]
            if covered:
                sum_cons = sum(deltas[c] for c in covered)
                net = deltas[p] + sum_cons
                explanation.append((p, covered, deltas[p], sum_cons, net))
                print(f"   • Pro '{p}' (+{deltas[p]:.1f}) couvre {covered} ({sum_cons:.1f}) → Net: +{net:.1f}")
        return True, explanation
    else:
        print("\nExplication (1-m) INFAISABLE.")
        return False, None

In [18]:
def explain_m_1(sol1_name, sol2_name):
    """Explication de type (m-1) : plusieurs Pros compensent un seul Con."""
    deltas, pros, cons = calculate_deltas(sol1_name, sol2_name)
    
    print(f"\n{'='*60}")
    print(f"EXPLICATION (m-1) : {sol1_name} > {sol2_name}")
    print(f"{'='*60}")
    print(f"Arguments Pour (Pros) : {len(pros)} critères")
    print(f"Arguments Contre (Cons) : {len(cons)} critères")
    
    if len(pros) == 0 or len(cons) == 0:
        if len(cons) == 0:
            print("\nAucun argument contre ! Solution 1 domine trivialement.")
            return True, "dominance"
        print("\nAucun argument pour - Explication impossible.")
        return False, None
    
    m = Model("Explication_m_1")
    
    x = m.addVars(pros, cons, vtype=GRB.BINARY, name="x")
    m.update()
    
    # Chaque Pro utilisé au plus une fois
    for p in pros:
        m.addConstr(quicksum(x[p, c] for c in cons) <= 1, name=f"Unique_{p}")
    
    # Chaque Con doit être compensé par un groupe de Pros
    epsilon = 0.001
    for c in cons:
        m.addConstr(deltas[c] + quicksum(deltas[p] * x[p, c] for p in pros) >= epsilon, name=f"Valid_{c}")
    
    m.setObjective(0, GRB.MINIMIZE)
    m.params.outputflag = 0
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        print("\nEXPLICATION (m-1) TROUVÉE :")
        explanation = []
        for c in cons:
            assigned = [p for p in pros if x[p, c].x > 0.5]
            sum_pros = sum(deltas[p] for p in assigned)
            net = deltas[c] + sum_pros
            explanation.append((c, assigned, deltas[c], sum_pros, net))
            print(f"   • Con '{c}' ({deltas[c]:.1f}) compensé par {assigned} (+{sum_pros:.1f}) → Net: +{net:.1f}")
        return True, explanation
    else:
        print("\nExplication (m-1) INFAISABLE.")
        return False, None

In [19]:
def explain_mixed(sol1_name, sol2_name):
    """Explication mixte combinant (1-m) et (m-1)."""
    deltas, pros, cons = calculate_deltas(sol1_name, sol2_name)
    
    print(f"\n{'='*60}")
    print(f"EXPLICATION MIXTE : {sol1_name} > {sol2_name}")
    print(f"{'='*60}")
    print(f"Arguments Pour (Pros) : {len(pros)} critères")
    print(f"Arguments Contre (Cons) : {len(cons)} critères")
    
    if len(pros) == 0 or len(cons) == 0:
        if len(cons) == 0:
            print("\nAucun argument contre ! Solution 1 domine trivialement.")
            return True, "dominance"
        print("\nAucun argument pour - Explication impossible.")
        return False, None
    
    m = Model("Explication_Mixte")
    
    # Variables de pivot
    z_1m = m.addVars(pros, vtype=GRB.BINARY, name="z_1m")  # p est pivot 1-m
    z_m1 = m.addVars(cons, vtype=GRB.BINARY, name="z_m1")  # c est pivot m-1
    
    # Variables de liaison
    v = m.addVars(pros, cons, vtype=GRB.BINARY, name="v")  # p couvre c en 1-m
    w = m.addVars(pros, cons, vtype=GRB.BINARY, name="w")  # p aide c en m-1
    
    m.update()
    
    # Chaque Con est soit pivot m-1, soit couvert en 1-m
    for c in cons:
        m.addConstr(z_m1[c] + quicksum(v[p, c] for p in pros) == 1, name=f"CoverCon_{c}")
    
    # Chaque Pro est soit pivot 1-m, soit support m-1 (ou rien)
    for p in pros:
        m.addConstr(z_1m[p] + quicksum(w[p, c] for c in cons) <= 1, name=f"UniquePro_{p}")
    
    # Consistance des liens
    for p in pros:
        for c in cons:
            m.addConstr(v[p, c] <= z_1m[p])  # v actif => p est pivot 1-m
            m.addConstr(w[p, c] <= z_m1[c])  # w actif => c est pivot m-1
    
    # Validité avec Big-M
    epsilon = 0.001
    BigM = 100000
    
    for p in pros:
        m.addConstr(deltas[p] + quicksum(deltas[c] * v[p, c] for c in cons) >= epsilon - BigM * (1 - z_1m[p]))
    
    for c in cons:
        m.addConstr(deltas[c] + quicksum(deltas[p] * w[p, c] for p in pros) >= epsilon - BigM * (1 - z_m1[c]))
    
    # Minimiser le nombre de trade-offs
    m.setObjective(quicksum(z_1m[p] for p in pros) + quicksum(z_m1[c] for c in cons), GRB.MINIMIZE)
    m.params.outputflag = 0
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        print("\nEXPLICATION MIXTE TROUVÉE :")
        explanation = {'1-m': [], 'm-1': []}
        
        for p in pros:
            if z_1m[p].x > 0.5:
                my_cons = [c for c in cons if v[p, c].x > 0.5]
                val = deltas[p] + sum(deltas[c] for c in my_cons)
                explanation['1-m'].append((p, my_cons, val))
                print(f"   [1-m] Pro '{p}' (+{deltas[p]:.1f}) couvre {my_cons} → Net: +{val:.1f}")
        
        for c in cons:
            if z_m1[c].x > 0.5:
                my_pros = [p for p in pros if w[p, c].x > 0.5]
                val = deltas[c] + sum(deltas[p] for p in my_pros)
                explanation['m-1'].append((c, my_pros, val))
                print(f"   [m-1] Con '{c}' ({deltas[c]:.1f}) compensé par {my_pros} → Net: +{val:.1f}")
        
        return True, explanation
    else:
        print("\nExplication mixte INFAISABLE.")
        return False, None

## 3. Analyse : Solution 1 vs Autres Solutions

Nous allons montrer que la **solution 1 est meilleure** que les solutions 2, 3 et 4.

In [20]:
def full_analysis(sol1_name, sol2_name):
    """Analyse complète pour comparer deux solutions."""
    print(f"\n{'#'*70}")
    print(f"#  ANALYSE COMPLÈTE : {sol1_name} EST MEILLEURE QUE {sol2_name}")
    print(f"{'#'*70}")
    
    deltas, pros, cons = calculate_deltas(sol1_name, sol2_name)
    
    # Résumé des deltas
    print(f"\nRÉSUMÉ DES CONTRIBUTIONS :")
    print(f"   Somme totale des deltas : {sum(deltas.values()):.2f}")
    print(f"   Nombre de Pros : {len(pros)}")
    print(f"   Nombre de Cons : {len(cons)}")
    print(f"   Critères neutres : {len([k for k, v in deltas.items() if v == 0])}")
    
    # Afficher top Pros et top Cons
    sorted_pros = sorted([(p, deltas[p]) for p in pros], key=lambda x: -x[1])
    sorted_cons = sorted([(c, deltas[c]) for c in cons], key=lambda x: x[1])
    
    print(f"\n   Top 5 Pros : {sorted_pros[:5]}")
    print(f"   Top 5 Cons (pires) : {sorted_cons[:5]}")
    
    results = {}
    
    # Test (1-1)
    success, expl = explain_1_1(sol1_name, sol2_name)
    results['1-1'] = success
    
    # Test (1-m)
    success, expl = explain_1_m(sol1_name, sol2_name)
    results['1-m'] = success
    
    # Test (m-1)
    success, expl = explain_m_1(sol1_name, sol2_name)
    results['m-1'] = success
    
    # Test Mixte
    success, expl = explain_mixed(sol1_name, sol2_name)
    results['mixte'] = success
    
    # Résumé
    print(f"\n{'='*70}")
    print(f"RÉSUMÉ DES EXPLICATIONS POUR : {sol1_name} > {sol2_name}")
    print(f"{'='*70}")
    for typ, success in results.items():
        status = "POSSIBLE" if success else "IMPOSSIBLE"
        print(f"   Explication ({typ}) : {status}")
    
    return results

### 3.1 Solution 1 vs Solution 2

In [21]:
results_1_vs_2 = full_analysis('solution 1', 'solution 2')


######################################################################
#  ANALYSE COMPLÈTE : solution 1 EST MEILLEURE QUE solution 2
######################################################################

RÉSUMÉ DES CONTRIBUTIONS :
   Somme totale des deltas : 2072.00
   Nombre de Pros : 13
   Nombre de Cons : 14
   Critères neutres : 0

   Top 5 Pros : [('n', 968), ('d', 926.0), ('z', 906.0), ('a', 876), ('b', 876)]
   Top 5 Cons (pires) : [('p', -900), ('e', -854.0), ('t', -826), ('r', -685.9999999999999), ('w', -582)]

EXPLICATION (1-1) : solution 1 > solution 2
Arguments Pour (Pros) : 13 critères
Arguments Contre (Cons) : 14 critères
Paires (1-1) valides : 128

Explication (1-1) INFAISABLE.

EXPLICATION (1-m) : solution 1 > solution 2
Arguments Pour (Pros) : 13 critères
Arguments Contre (Cons) : 14 critères

EXPLICATION (1-m) TROUVÉE :
   • Pro 'a' (+876.0) couvre ['k', 'x'] (-730.0) → Net: +146.0
   • Pro 'b' (+876.0) couvre ['q'] (-514.0) → Net: +362.0
   • Pro 'c' (+778.0) couv

### 3.2 Solution 1 vs Solution 3

In [22]:
results_1_vs_3 = full_analysis('solution 1', 'solution 3')


######################################################################
#  ANALYSE COMPLÈTE : solution 1 EST MEILLEURE QUE solution 3
######################################################################

RÉSUMÉ DES CONTRIBUTIONS :
   Somme totale des deltas : 35456.60
   Nombre de Pros : 20
   Nombre de Cons : 7
   Critères neutres : 0

   Top 5 Pros : [('c', 7120), ('z', 6231.0), ('f', 4188), ('r', 3624), ('e', 3464)]
   Top 5 Cons (pires) : [('k', -2112), ('i', -1560), ('p', -1536), ('v', -1348), ('y', -1330)]

EXPLICATION (1-1) : solution 1 > solution 3
Arguments Pour (Pros) : 20 critères
Arguments Contre (Cons) : 7 critères
Paires (1-1) valides : 87

EXPLICATION (1-1) TROUVÉE :
   • Trade-off (c,k) : +7120.0 + (-2112.0) = +5008.0
   • Trade-off (d,l) : +690.0 + (-634.0) = +56.0
   • Trade-off (m,i) : +2554.0 + (-1560.0) = +994.0
   • Trade-off (n,y) : +2508.0 + (-1330.0) = +1178.0
   • Trade-off (q,v) : +1606.0 + (-1348.0) = +258.0
   • Trade-off (u,b) : +1160.0 + (-532.0) = +628

### 3.3 Solution 1 vs Solution 4

In [23]:
results_1_vs_4 = full_analysis('solution 1', 'solution 4')


######################################################################
#  ANALYSE COMPLÈTE : solution 1 EST MEILLEURE QUE solution 4
######################################################################

RÉSUMÉ DES CONTRIBUTIONS :
   Somme totale des deltas : 4014.00
   Nombre de Pros : 14
   Nombre de Cons : 13
   Critères neutres : 0

   Top 5 Pros : [('h', 4676.0), ('k', 4560), ('w', 4236), ('z', 3590.0), ('i', 2592)]
   Top 5 Cons (pires) : [('r', -4812), ('l', -3990.0), ('t', -3766), ('p', -1816), ('c', -1750)]

EXPLICATION (1-1) : solution 1 > solution 4
Arguments Pour (Pros) : 14 critères
Arguments Contre (Cons) : 13 critères
Paires (1-1) valides : 89

Explication (1-1) INFAISABLE.

EXPLICATION (1-m) : solution 1 > solution 4
Arguments Pour (Pros) : 14 critères
Arguments Contre (Cons) : 13 critères

Explication (1-m) INFAISABLE.

EXPLICATION (m-1) : solution 1 > solution 4
Arguments Pour (Pros) : 14 critères
Arguments Contre (Cons) : 13 critères

Explication (m-1) INFAISABLE.


## 4. Conclusion

In [28]:
print("\n" + "="*70)
print("CONCLUSION FINALE : LA SOLUTION 1 EST LA MEILLEURE")
print("="*70)

all_results = {
    'vs Solution 2': results_1_vs_2,
    'vs Solution 3': results_1_vs_3,
    'vs Solution 4': results_1_vs_4
}

print("\nTABLEAU RÉCAPITULATIF DES EXPLICATIONS :")
print("-" * 60)
print(f"{'Comparaison':<20} {'(1-1)':<12} {'(1-m)':<12} {'(m-1)':<12} {'Mixte':<12}")
print("-" * 60)

for comp, res in all_results.items():
    r1 = "POSSIBLE" if res.get('1-1') else "IMPOSSIBLE"
    r2 = "POSSIBLE" if res.get('1-m') else "IMPOSSIBLE"
    r3 = "POSSIBLE" if res.get('m-1') else "IMPOSSIBLE"
    r4 = "POSSIBLE" if res.get('mixte') else "IMPOSSIBLE"
    print(f"{comp:<20} {r1:<12} {r2:<12} {r3:<12} {r4:<12}")


CONCLUSION FINALE : LA SOLUTION 1 EST LA MEILLEURE

TABLEAU RÉCAPITULATIF DES EXPLICATIONS :
------------------------------------------------------------
Comparaison          (1-1)        (1-m)        (m-1)        Mixte       
------------------------------------------------------------
vs Solution 2        IMPOSSIBLE   POSSIBLE     IMPOSSIBLE   POSSIBLE    
vs Solution 3        POSSIBLE     POSSIBLE     POSSIBLE     POSSIBLE    
vs Solution 4        IMPOSSIBLE   IMPOSSIBLE   IMPOSSIBLE   POSSIBLE    
